***

## Exact 3-SAT

>CSC427, semester 202 (jan-may 2020)
<br>
10 April 2020
<br><br>
burton rosenberg
<br>
univ of miami
<br>
(c) 2020 all rights reserved

***


A famous NP-complete problem is 3-SAT. It is a problem in finding an assignment of truth values to 
variables so that a certain logical formula evaluates true. The logical formula has a restricted 
format. It must be in _Conjunctive Normal Form_, and furthermore, each conjunct must have exactly 
three terms.

When a formula is in _Conjunctive Normal Form_ it is an "and" (conjunction) over a collection of
clauses, and each clause is an "or" (disjunction) over a collection of variables or their negations.
A simple example is,

$$
(a \lor b \lor \neg c) \land ( \neg a \lor e \lor \neg f) \land ( f \lor \neg a \lor c)
$$

Note that this example is actually in our 3-SAT form, as each of the clauses has exactly three
variables.

A _satisfying assignment_ is a way of assigning values to the variables so that the formula
calculates out to true. This means that for each of the clauses, at least one of the three
variables must be either not negatated and the variable assigned true, or negated and the variable
assigned false.

The above formula is true when a is set true, f is set false and c is set true.

### 3-SAT is in NP

Given a satisfying assignment it is polynomial time to verify that the formula calculates out
to true with that assignment. The code to do this is in the Exact3SAT class, the verify method. 
You can work out the run time and make sure you understand it is polynomial time.

One can find a satisfying assignmentby bruit-force - trying every truth combination for
all the variables in the formula. There are an exponential number of possibilities for truth
assignments, but if the immense amount of time is not an issue, an assignment will be found, 
if one exists, and the search will definitively halt with a negative result if no truth assignment
exists.

The search solution adopted here is not much better than bruit search. The search scans the clauses
in the order presented, building up a collection of truth assignments to make each clause true as
it is encountered.

The code uses recursion based on attempting to make a particular clause evaluate true, indicated
by the variable in the code "clause_number", and the recurse in clause_number+1. The result from the
recursion is either True, meaning the recursion managed to complete the variable assignment to make
clause clause_number+1 and all following clauses evaluate true, or returns False, meaning it could not.

If True, since the clause at clause_number has been made to evaluate true, true is returned.

If False, then a different way to make clause_number true is tried. If after trying all ways to make
clause_number true, and for none of them does the recursive call for clause_number+1 returns True, return
False.
 
   
 

In [72]:
###
### exact 3-SAT solver
### author: bjr
### date: 10 apr 2020
###

# last update: 10 apr 2020


# problem statement: 

# given a set of clauses, where is clause is three un-negated or negated variables
# find a truth assignment to the variables such that every clause has at least one 
# member being true - i.e. an un-negated variable set to True, or a negated variable
# set to False

# this is called a statisfying assignment. 


class Exact3SAT:

    # data structures

    # clause list:
    #   clause := [(v_1,0|1), (v_2,0|1), (v_3,0|1)]
    #   where v_i are string type variable names, and 0 for unnegated, 1 for negated
    #   clause_list := [ clause, clause, .., clause ]

    # value dictionary:
    # a dictionary of value assignents { .... variable | (True|False)  ...}

    def __init__(self,clause_list,verbose=0):
        self.values = {}
        self.clause_list = clause_list
        self.clause_list_len = len(clause_list)
        self.verbose = verbose

    def get_assignment(self):
        return self.values

    def is_satisfied(self,clause):
        return self.is_satisfied_aux(clause,self.values)
        
    def is_satisfied_aux(self,clause,values):
       
        for var in clause:
            if var[0] in values:
                if values[var[0]]==True and var[1]==0:
                    if self.verbose>0: print("is_satisfied: RETURNS True,\n\tinput", clause, values )
                    return True
                if values[var[0]]==False and var[1]==1:
                    if self.verbose>0: print("is_satisfied: RETURNS True,\n\tinput", clause, values )
                    return True
        if self.verbose>0: print("is_satisfied: RETURNS False,\n\tinput", clause, values )
        return False
    
    def solve(self):
        return self.search_aux(0)

    def search_aux(self,clause_number):
        if self.verbose>0: print('search_aux: ENTERED (clause number, values)\n\t', clause_number, self.values)
        
        if clause_number>=self.clause_list_len:
            return True
            
        clause = self.clause_list[clause_number] 
        if self.is_satisfied(clause):
            return self.search_aux(clause_number+1)

        for j in range(3):
            var = clause[j][0]
            if var not in self.values:
                self.values[var] = True if clause[j][1] == 0 else False
                if self.verbose>1: print("search_aux: TRYING a setting (clause, var, setting)\n\t",j,var,self.values[var])
                if self.search_aux(clause_number+1):
                    return True
                del self.values[var]

        # could not find a variable to set
        return False    
        
    def verify(self,cnf,values):
        for clause in cnf:
            if not self.is_satisfied_aux(clause,values):
                return False
        return True
            
        
    

### Test functions



In [73]:
from random import randint 

class Exact3SAT_Test:
    
    @staticmethod
    def run_3sat(cnf,verbose=0):
        three_sat = Exact3SAT(cnf,verbose)
        result = three_sat.solve()
        assignment = three_sat.get_assignment()
        print(cnf, "\n\t", result, assignment)
        # check
        assert( result == three_sat.verify(cnf,assignment))

    @staticmethod
    def random_cnf3_3var(n):
        l = []
        for i in range(n):
            l += [[('x',randint(0,1)),('y',randint(0,1)),('z',randint(0,1))]]
        return l

    @staticmethod
    def random_cnf3_4var(n):
        l = []
        var_name = [ 'x', 'y', 'z', 't' ]
        for i in range(n):
            l += [[
                    (var_name[randint(0,3)],randint(0,1)),
                    (var_name[randint(0,3)],randint(0,1)),
                    (var_name[randint(0,3)],randint(0,1))
                 ]]
        return l

    @staticmethod
    def bruit_force_3var(three_sat, cnf):
        # try to solve it by bruit force
        for x in range(2):
            for y in range(2):
                for z in range(2):
                    values = { 'x':bool(x), 'y':bool(y), 'z':bool(z) }
                    if three_sat.verify(cnf,values):
                        return values
        return {}

    @staticmethod
    def bruit_force_4var(three_sat, cnf):
        # try to solve it by bruit force
        for x in range(2):
            for y in range(2):
                for z in range(2):
                    for t in range(2):
                        values = { 'x':bool(x), 'y':bool(y), 'z':bool(z), 't':bool(t) }
                        if three_sat.verify(cnf,values):
                            return values
        return {}

    @staticmethod
    def fuzz_3sat_3var(n,m,verbose=0):

        verdict = True
        for i in range(n):
            random_cnf = Exact3SAT_Test.random_cnf3_3var(m)
            three_sat = Exact3SAT(random_cnf,verbose)
            result = three_sat.solve()
            assignment = three_sat.get_assignment()
            if verbose>0 : print(random_cnf, "\n\t", result, assignment)
            if result:
                v = three_sat.verify(random_cnf, assignment)
                verdict  = verdict and v
                if verbose>0 : print("\tverified:", v)
            else:
                v = Exact3SAT_Test.bruit_force_3var(three_sat, random_cnf)
                verdict = verdict and len(v)==0
                if verbose>0 : print("\tbruit force: ", v)
        return verdict

    @staticmethod
    def fuzz_3sat_4var(n,m,verbose=0):
        
        verdict = True
        for i in range(n):
            random_cnf = Exact3SAT_Test.random_cnf3_4var(m)
            three_sat = Exact3SAT(random_cnf,verbose)
            result = three_sat.solve()
            assignment = three_sat.get_assignment()
            if verbose>0 : print(random_cnf, "\n\t", result, assignment)
            if result:
                v = three_sat.verify(random_cnf, assignment)
                verdict  = verdict and v
                if verbose>0 : print("\tverified:", v)
            else:
                v = Exact3SAT_Test.bruit_force_4var(three_sat, random_cnf)
                verdict = verdict and len(v)==0
                if verbose>0 : print("\tbruit force: ", v)
        return verdict
            



### Run tests


1. A few hand crafted CNF's are tried.
2. Random 3-SAT's are generated over variables x, y and z. The 3-SAT solver solves the random 3-SAT, and
   the result is verified either by varifing a positive result, or bruit-search for a solution to confirm
   a negative result.
3. Random 3-SAT's are generated over the four variables x, y, z and t, and the test is the same as above.



In [75]:
            
cnf3_1 = [ [('x',0),('y',0),('z',0)] ]
cnf3_2 = [ [('x',1),('y',0),('z',0)] ]
cnf3_3 = [ [('x',0),('y',0),('z',0)], [('x',1),('y',0),('z',0)] ]
cnf3_4 = [ [('x',1),('y',0),('z',0)], [('x',0),('y',1),('z',0)] ]
cnf3_5 = [ [('x',0),('x',0),('x',0)], [('x',1),('x',1),('x',1)] ]
cnf3_6 = [ [('x',0),('x',0),('x',0)], [('y',1),('y',1),('y',1)], 
           [('x',1),('y',0),('z',0)], [('z',1),('z',1),('z',1)] ]
cnf3_7 = [ [('x',0),('x',0),('x',0)], [('y',1),('y',1),('y',1)], 
           [('x',1),('y',0),('z',0)], [('x',0),('z',1),('z',1)] ]
 
cnfs = [ cnf3_1, cnf3_2, cnf3_3, cnf3_4, cnf3_5, cnf3_6, cnf3_7]

for cnf in cnfs:
    Exact3SAT_Test.run_3sat(cnf)
    
print("Test: fuzz_3sat_3var\n\tverdict = ", Exact3SAT_Test.fuzz_3sat_3var(7,20))
print("Test: fuzz_3sat_4var\n\tverdict = ", Exact3SAT_Test.fuzz_3sat_4var(7,20))

[[('x', 0), ('y', 0), ('z', 0)]] 
	 True {'x': True}
[[('x', 1), ('y', 0), ('z', 0)]] 
	 True {'x': False}
[[('x', 0), ('y', 0), ('z', 0)], [('x', 1), ('y', 0), ('z', 0)]] 
	 True {'x': True, 'y': True}
[[('x', 1), ('y', 0), ('z', 0)], [('x', 0), ('y', 1), ('z', 0)]] 
	 True {'x': False, 'y': False}
[[('x', 0), ('x', 0), ('x', 0)], [('x', 1), ('x', 1), ('x', 1)]] 
	 False {}
[[('x', 0), ('x', 0), ('x', 0)], [('y', 1), ('y', 1), ('y', 1)], [('x', 1), ('y', 0), ('z', 0)], [('z', 1), ('z', 1), ('z', 1)]] 
	 False {}
[[('x', 0), ('x', 0), ('x', 0)], [('y', 1), ('y', 1), ('y', 1)], [('x', 1), ('y', 0), ('z', 0)], [('x', 0), ('z', 1), ('z', 1)]] 
	 True {'x': True, 'y': False, 'z': True}
Test: fuzz_3sat_3var
	verdict =  True
Test: fuzz_3sat_4var
	verdict =  True


### Reducing general SAT to 3-SAT.

3-SAT is NP-complete. Meaning that it is first in NP; and second, any problem in NP can be turned into a problem in 3-SAT. 

An example is the general SAT problem for Conjunctive Normal Form, where the clauses are not restricted to exactly tree variables. There could be greater, or fewer.

Your project is to write a reduction so that a general SAT problem turns into a 3-SAT problem, such that solving that 3-SAT gives a solution to the original SAT. At the same time, if the original SAT as no solution, also the 3-SAT have no solution.

The general theory of NP-completeness means that any problem in NP can have a reduction to 3-SAT, often first by reducing it to SAT. The intuitive reason that any problme in NP can be reduced to SAT, is that in the end, any problem that a computer can solve is really about an enormous boolean formula for which truth values are needed so that the formula comes out right. In the end, that is all a computer does. Find satisfying assignments to big logical formulas.

From there the logical formulas must be transformed into SAT, and then into 3-SAT. The composite then is that the original problem is not a problem in 3-SAT solvable if an if the original problem was solvable, and often the satisfiying assignment (if any) can be read backwards to give the solution to the original problem.

